# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686577


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:04<01:04,  2.22s/it]

Rendering models:  13%|█▎        | 4/31 [00:07<00:52,  1.96s/it]

Rendering models:  23%|██▎       | 7/31 [00:07<00:33,  1.39s/it]

Rendering models:  29%|██▉       | 9/31 [00:07<00:21,  1.01it/s]

...

Rendering models:  71%|███████   | 22/31 [00:08<00:01,  5.68it/s]

Rendering models:  77%|███████▋  | 24/31 [00:08<00:01,  6.72it/s]

Rendering models:  87%|████████▋ | 27/31 [00:09<00:00,  8.30it/s]

Rendering models:  94%|█████████▎| 29/31 [00:09<00:00,  9.32it/s]

hailstorm3812                         0.000897
shirogane                             0.000650
Dynamatt                              0.000527
not-logged-in-742cef5e1a82d2ddd0fb    0.000567
mtajnai                               0.001231
nsyee                                 0.000392
justsomenoodles                       0.001235
jmfranci                              0.002500
insert_name                           0.000664
ElisabethB                            0.000371
not-logged-in-5704ea8f3a0f12b5bfb2    0.000789
acapirala                             0.001287
Lavadude                              0.238337
Lavadude                              0.064186
karl_stark                            0.009569
not-logged-in-704ca07c4131cfda3ea2    0.002137
not-logged-in-15498ebaa693dd5a198d    0.000392
not-logged-in-8700ff8251e0704f745c    0.021354
ShirleyL                              0.000480
not-logged-in-2027f1f6703c19b00bd8    0.006852
GrantCrooks                           0.001315
nyoung55     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())